## Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.


### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.


#### CNN-RNN Architecture Overview
1. The model used is a CNN-RNN architecthure
2. The CNN works as the enocder and the RNN works as the Decoder
3. The CNN-Enocder tries to encode the image into feature-encodings
4. The RNN-Decoder tries to decode the feature-encodings from CNN-Enocder into captions
5. The CNN feature-enocdings goes as the input at first time to the RNN-Decoder
6. You can see this in the below image: 
![Image Captioning CNN-RNN model](images/encoder-decoder.png)

#### CNN-Enocder ResNet50
1. The CNN-Enocder used is ResNet50 model
2. The ResNet50 used was a pre-trained model on ImageNet and the weights were loaded from this pre-trained weights
3. CNN-Enocder layers were not trainable
4. The ResNet50 model consists of 50 layers 
5. The building block of ResNet50 is residual block
6. Below is the diagram of residual block:
![Image Captioning RESIDUAL BLOCK model](images/residual-block.png)
7. Instead of hoping each stack of layers directly fits a desired underlying mapping, we explicitly let these layers fit a residual mapping. 
8. The original mapping is recast into F(x)+x. 
9. We hypothesize that it is easier to optimize the residual mapping than to optimize the original, unreferenced mapping. 
10. To the extreme, if an identity mapping were optimal, it would be easier to push the residual to zero than to fit an identity mapping by a stack of nonlinear layers.
11. To know more about Residual Networks, refer the paper [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf).



#### RNN-Decoder
1] The RNN-Decoder:  

| Layer         		|     Description	        					|
|:---------------------:|:---------------------------------------------:|
| Input         		| Captions encoded into numbers     		    | 
| Embedding Layer     	| 128 Neurons                	                |
| LSTM Layer			| 100 Neurons 									|
| Output Layer      	| 414113 Neurons	                            |  
2] EMBEDDING LAYER:
  - When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. 
  - Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other values set to 0.
  - The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation.
![Image Captioning Embedding_Explainataion_1](images/embedding_explainataion_1.png)
  - To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings.
  - Embeddings are just a fully connected layer like you've seen before.
  - We call this layer the embedding layer and the weights are embedding weights.
  - We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix.
  - We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.
  - Instead of doing the matrix multiplication, we use the weight matrix as a lookup table.
![Image Captioning Embedding_Explainataion_2](images/embedding_lookup.png)

3] LSTM Layer:
  - LSTM layer comprises of LSTM Nodes
  - Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies
  - LSTMs are explicitly designed to avoid the long-term dependency problem.
  - Below is LSTM Node Diagram and the computations that happend in LSTM node
  ![Image Captioning LSTM Node](images/lstm_node.png)



### Transformation Chosen and Why

1. `transform_train` is a part of pre-processing step
2. The original image is resized to 256
3. The Resized image gets randomly cropped to 224x224
4. The Cropped Image has probability of 50% of horizontally flip
5. The Image is Converted to Torch Tensor. This is necessary so that image (H x W x C) is converted to (C x H x W) which can be acccepted by the Model
6. Normalize the Torch Tensor image with mean and standard deviation.
7. The transform_train in code : `transform_train = transforms.Compose([transforms.Resize(256), transforms.RandomCrop(224), transforms.RandomHorizontalFlip(), transforms.ToTensor(), (0.229, 0.224, 0.225))])`
8. I left the transform at it's provided value
9. I think it's a good transform because every epoch the model will see a different image with the same captions
10. This will help the model to generalize by looking different and more data
11. The images in the dataset is of heights and widths and the model can not accept images of varying length and hence all are resized and cropped to 224x224 hieght and width
12. If using a pre-trained model, you must perform the corresponding appropriate normalization.

### Trainable Parameters Chosen and Why

1. Hyper-parameters selected:
   - number of epochs = 3
   - batch size = 10
   - embedding nodes = 128
   - lstm nodes = 100
   - optimizer = adam
2. Trainable weights were only of RNN-Decoder and The Last layer of CNN-Encoder
3. `params = list(decoder.parameters()) + list(encoder.embed.parameters())`
4. The ResNet50 is already a pretrained model on ImageNet and hence is a good candidate for extracting rich features... By not training the whole model of ResNet50 save training time and precious GPU memory... Because training ResNet50 which is a humongous network will be really costly
5. The last layer of CNN Encoder is trainable so that the features selected are based on this specific dataset
6. By this stratergy we pick the better features and save Gpu memory and Training time
7. RNN-Decoder weights are trainable because it is necessary for them to learn the context of images and captions.


### Optimizer

- Adam optimizer is selected
- Adam is chosen because it is computational effficient
- Faster convergence
- Much better than other variants of gradient descent

In [3]:
import sys
sys.path.append('./cocoapi/PythonAPI')
from pycocotools.coco import COCO
!pip install nltk
import nltk
nltk.download('punkt')
import torch
import torch.nn as nn
from torchvision import transforms
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 3          # batch size
vocab_threshold = 5       # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 128          # dimensionality of image and word embeddings
hidden_size = 100        # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.0001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to /home/jai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  1%|          | 2643/414113 [00:00<00:31, 13249.69it/s]

Done (t=0.42s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:29<00:00, 14013.36it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions[:, :-1])
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/138038], Loss: 8.5686, Perplexity: 5263.7438
Epoch [1/3], Step [200/138038], Loss: 6.6477, Perplexity: 771.03467
Epoch [1/3], Step [300/138038], Loss: 5.2150, Perplexity: 184.01379
Epoch [1/3], Step [400/138038], Loss: 5.6655, Perplexity: 288.7222
Epoch [1/3], Step [500/138038], Loss: 5.0063, Perplexity: 149.3552
Epoch [1/3], Step [600/138038], Loss: 4.8283, Perplexity: 125.0007
Epoch [1/3], Step [700/138038], Loss: 5.7179, Perplexity: 304.2780
Epoch [1/3], Step [800/138038], Loss: 4.6182, Perplexity: 101.3149
Epoch [1/3], Step [900/138038], Loss: 5.3391, Perplexity: 208.3182
Epoch [1/3], Step [1000/138038], Loss: 4.6895, Perplexity: 108.7992
Epoch [1/3], Step [1100/138038], Loss: 4.2879, Perplexity: 72.81400
Epoch [1/3], Step [1200/138038], Loss: 5.0057, Perplexity: 149.2591
Epoch [1/3], Step [1300/138038], Loss: 4.7591, Perplexity: 116.6428
Epoch [1/3], Step [1400/138038], Loss: 4.3322, Perplexity: 76.11296
Epoch [1/3], Step [1500/138038], Loss: 4.2645, Perplex

Epoch [1/3], Step [12100/138038], Loss: 4.0432, Perplexity: 57.00975
Epoch [1/3], Step [12200/138038], Loss: 3.3443, Perplexity: 28.34206
Epoch [1/3], Step [12300/138038], Loss: 4.5519, Perplexity: 94.81028
Epoch [1/3], Step [12400/138038], Loss: 3.3265, Perplexity: 27.84173
Epoch [1/3], Step [12500/138038], Loss: 3.6667, Perplexity: 39.12309
Epoch [1/3], Step [12600/138038], Loss: 4.5171, Perplexity: 91.56865
Epoch [1/3], Step [12700/138038], Loss: 3.5817, Perplexity: 35.93408
Epoch [1/3], Step [12800/138038], Loss: 3.6377, Perplexity: 38.00561
Epoch [1/3], Step [12900/138038], Loss: 4.4117, Perplexity: 82.41141
Epoch [1/3], Step [13000/138038], Loss: 3.4575, Perplexity: 31.73709
Epoch [1/3], Step [13100/138038], Loss: 4.0240, Perplexity: 55.92453
Epoch [1/3], Step [13200/138038], Loss: 3.5956, Perplexity: 36.43741
Epoch [1/3], Step [13300/138038], Loss: 4.3064, Perplexity: 74.16936
Epoch [1/3], Step [13400/138038], Loss: 3.7802, Perplexity: 43.82321
Epoch [1/3], Step [13500/138038], 

Epoch [1/3], Step [23900/138038], Loss: 3.2845, Perplexity: 26.69646
Epoch [1/3], Step [24000/138038], Loss: 3.5984, Perplexity: 36.54088
Epoch [1/3], Step [24100/138038], Loss: 4.9037, Perplexity: 134.7816
Epoch [1/3], Step [24200/138038], Loss: 3.0545, Perplexity: 21.209812
Epoch [1/3], Step [24300/138038], Loss: 2.8637, Perplexity: 17.52710
Epoch [1/3], Step [24400/138038], Loss: 2.9109, Perplexity: 18.37307
Epoch [1/3], Step [24500/138038], Loss: 3.8301, Perplexity: 46.06902
Epoch [1/3], Step [24600/138038], Loss: 4.4488, Perplexity: 85.52164
Epoch [1/3], Step [24700/138038], Loss: 3.2617, Perplexity: 26.09405
Epoch [1/3], Step [24800/138038], Loss: 3.5144, Perplexity: 33.59662
Epoch [1/3], Step [24900/138038], Loss: 3.8400, Perplexity: 46.52477
Epoch [1/3], Step [25000/138038], Loss: 3.5025, Perplexity: 33.19805
Epoch [1/3], Step [25100/138038], Loss: 4.9707, Perplexity: 144.1274
Epoch [1/3], Step [25200/138038], Loss: 4.2295, Perplexity: 68.68360
Epoch [1/3], Step [25300/138038],

Epoch [1/3], Step [35700/138038], Loss: 2.8109, Perplexity: 16.62566
Epoch [1/3], Step [35800/138038], Loss: 2.8033, Perplexity: 16.49872
Epoch [1/3], Step [35900/138038], Loss: 3.7450, Perplexity: 42.30819
Epoch [1/3], Step [36000/138038], Loss: 2.8795, Perplexity: 17.80564
Epoch [1/3], Step [36100/138038], Loss: 3.2155, Perplexity: 24.91511
Epoch [1/3], Step [36200/138038], Loss: 3.3663, Perplexity: 28.96993
Epoch [1/3], Step [36300/138038], Loss: 3.8657, Perplexity: 47.7359
Epoch [1/3], Step [36400/138038], Loss: 3.0360, Perplexity: 20.82201
Epoch [1/3], Step [36500/138038], Loss: 3.1364, Perplexity: 23.02061
Epoch [1/3], Step [36600/138038], Loss: 4.2066, Perplexity: 67.12641
Epoch [1/3], Step [36700/138038], Loss: 3.6267, Perplexity: 37.58870
Epoch [1/3], Step [36800/138038], Loss: 3.7455, Perplexity: 42.33155
Epoch [1/3], Step [36900/138038], Loss: 4.1058, Perplexity: 60.69061
Epoch [1/3], Step [37000/138038], Loss: 4.7067, Perplexity: 110.6813
Epoch [1/3], Step [37100/138038], L

Epoch [1/3], Step [47500/138038], Loss: 2.8126, Perplexity: 16.65302
Epoch [1/3], Step [47600/138038], Loss: 2.7977, Perplexity: 16.40741
Epoch [1/3], Step [47700/138038], Loss: 2.4577, Perplexity: 11.67811
Epoch [1/3], Step [47800/138038], Loss: 3.1066, Perplexity: 22.3441
Epoch [1/3], Step [47900/138038], Loss: 2.8816, Perplexity: 17.84365
Epoch [1/3], Step [48000/138038], Loss: 2.7723, Perplexity: 15.99600
Epoch [1/3], Step [48100/138038], Loss: 2.6535, Perplexity: 14.20391
Epoch [1/3], Step [48200/138038], Loss: 3.4741, Perplexity: 32.26932
Epoch [1/3], Step [48300/138038], Loss: 2.4853, Perplexity: 12.00468
Epoch [1/3], Step [48400/138038], Loss: 2.1443, Perplexity: 8.536191
Epoch [1/3], Step [48500/138038], Loss: 4.8910, Perplexity: 133.0816
Epoch [1/3], Step [48600/138038], Loss: 2.9137, Perplexity: 18.42464
Epoch [1/3], Step [48700/138038], Loss: 4.4796, Perplexity: 88.20241
Epoch [1/3], Step [48800/138038], Loss: 3.1966, Perplexity: 24.44820
Epoch [1/3], Step [48900/138038], L

...

Epoch [3/3], Step [80500/138038], Loss: 2.9602, Perplexity: 19.3013
Epoch [3/3], Step [80600/138038], Loss: 3.7757, Perplexity: 43.62936
Epoch [3/3], Step [80700/138038], Loss: 2.8486, Perplexity: 17.2634
Epoch [3/3], Step [80800/138038], Loss: 2.6020, Perplexity: 13.4913
Epoch [3/3], Step [80900/138038], Loss: 3.4396, Perplexity: 31.1739
Epoch [3/3], Step [81000/138038], Loss: 2.0341, Perplexity: 7.64552
Epoch [3/3], Step [81100/138038], Loss: 2.5351, Perplexity: 12.61790
Epoch [3/3], Step [81200/138038], Loss: 2.0818, Perplexity: 8.01933
Epoch [3/3], Step [81300/138038], Loss: 1.7353, Perplexity: 5.67094
Epoch [3/3], Step [81400/138038], Loss: 2.7408, Perplexity: 15.50016
Epoch [3/3], Step [81500/138038], Loss: 2.8762, Perplexity: 17.74742
Epoch [3/3], Step [81600/138038], Loss: 2.1991, Perplexity: 9.01652
Epoch [3/3], Step [81700/138038], Loss: 1.8617, Perplexity: 6.43495
Epoch [3/3], Step [81800/138038], Loss: 1.3929, Perplexity: 4.026698
Epoch [3/3], Step [81900/138038], Loss: 2.0

Epoch [3/3], Step [92500/138038], Loss: 3.1656, Perplexity: 23.7034
Epoch [3/3], Step [92600/138038], Loss: 1.9836, Perplexity: 7.26897
Epoch [3/3], Step [92700/138038], Loss: 2.7907, Perplexity: 16.2924
Epoch [3/3], Step [92800/138038], Loss: 3.4324, Perplexity: 30.9521
Epoch [3/3], Step [92900/138038], Loss: 2.1071, Perplexity: 8.22451
Epoch [3/3], Step [93000/138038], Loss: 2.7586, Perplexity: 15.7785
Epoch [3/3], Step [93100/138038], Loss: 2.1748, Perplexity: 8.80052
Epoch [3/3], Step [93200/138038], Loss: 2.1459, Perplexity: 8.549498
Epoch [3/3], Step [93300/138038], Loss: 2.5813, Perplexity: 13.21476
Epoch [3/3], Step [93400/138038], Loss: 3.0215, Perplexity: 20.5218
Epoch [3/3], Step [93500/138038], Loss: 1.7904, Perplexity: 5.99174
Epoch [3/3], Step [93600/138038], Loss: 2.4320, Perplexity: 11.3821
Epoch [3/3], Step [93700/138038], Loss: 2.8603, Perplexity: 17.46664
Epoch [3/3], Step [93800/138038], Loss: 3.5226, Perplexity: 33.87334
Epoch [3/3], Step [93900/138038], Loss: 1.88

Epoch [3/3], Step [104400/138038], Loss: 2.6964, Perplexity: 14.8267
Epoch [3/3], Step [104500/138038], Loss: 2.9104, Perplexity: 18.3633
Epoch [3/3], Step [104600/138038], Loss: 2.1864, Perplexity: 8.90275
Epoch [3/3], Step [104700/138038], Loss: 2.9735, Perplexity: 19.5612
Epoch [3/3], Step [104800/138038], Loss: 3.5560, Perplexity: 35.0225
Epoch [3/3], Step [104900/138038], Loss: 2.5002, Perplexity: 12.1852
Epoch [3/3], Step [105000/138038], Loss: 2.6323, Perplexity: 13.9058
Epoch [3/3], Step [105100/138038], Loss: 3.0775, Perplexity: 21.7035
Epoch [3/3], Step [105200/138038], Loss: 2.6850, Perplexity: 14.6575
Epoch [3/3], Step [105300/138038], Loss: 2.7760, Perplexity: 16.05529
Epoch [3/3], Step [105400/138038], Loss: 2.0775, Perplexity: 7.98472
Epoch [3/3], Step [105500/138038], Loss: 2.2336, Perplexity: 9.33368
Epoch [3/3], Step [105600/138038], Loss: 2.7395, Perplexity: 15.4798
Epoch [3/3], Step [105700/138038], Loss: 2.1162, Perplexity: 8.29929
Epoch [3/3], Step [105800/138038]

Epoch [3/3], Step [116200/138038], Loss: 2.3896, Perplexity: 10.9088
Epoch [3/3], Step [116300/138038], Loss: 3.0512, Perplexity: 21.1408
Epoch [3/3], Step [116400/138038], Loss: 4.3008, Perplexity: 73.7593
Epoch [3/3], Step [116500/138038], Loss: 2.6851, Perplexity: 14.6599
Epoch [3/3], Step [116600/138038], Loss: 2.8647, Perplexity: 17.5434
Epoch [3/3], Step [116700/138038], Loss: 2.0040, Perplexity: 7.41887
Epoch [3/3], Step [116800/138038], Loss: 2.7541, Perplexity: 15.70725
Epoch [3/3], Step [116900/138038], Loss: 3.0219, Perplexity: 20.5293
Epoch [3/3], Step [117000/138038], Loss: 2.9881, Perplexity: 19.84803
Epoch [3/3], Step [117100/138038], Loss: 3.0766, Perplexity: 21.6835
Epoch [3/3], Step [117200/138038], Loss: 1.5370, Perplexity: 4.650532
Epoch [3/3], Step [117300/138038], Loss: 2.6679, Perplexity: 14.4101
Epoch [3/3], Step [117400/138038], Loss: 2.1249, Perplexity: 8.372249
Epoch [3/3], Step [117500/138038], Loss: 4.0636, Perplexity: 58.1814
Epoch [3/3], Step [117600/1380

Epoch [3/3], Step [128000/138038], Loss: 2.1731, Perplexity: 8.785794
Epoch [3/3], Step [128100/138038], Loss: 3.4485, Perplexity: 31.4527
Epoch [3/3], Step [128200/138038], Loss: 2.6992, Perplexity: 14.8679
Epoch [3/3], Step [128300/138038], Loss: 3.1350, Perplexity: 22.98910
Epoch [3/3], Step [128400/138038], Loss: 3.2690, Perplexity: 26.2851
Epoch [3/3], Step [128500/138038], Loss: 2.2970, Perplexity: 9.943987
Epoch [3/3], Step [128600/138038], Loss: 2.3822, Perplexity: 10.8283
Epoch [3/3], Step [128700/138038], Loss: 2.8874, Perplexity: 17.94601
Epoch [3/3], Step [128800/138038], Loss: 1.8701, Perplexity: 6.48936
Epoch [3/3], Step [128900/138038], Loss: 3.4509, Perplexity: 31.5277
Epoch [3/3], Step [129000/138038], Loss: 3.0091, Perplexity: 20.2693
Epoch [3/3], Step [129100/138038], Loss: 3.1505, Perplexity: 23.3478
Epoch [3/3], Step [129200/138038], Loss: 2.3955, Perplexity: 10.9734
Epoch [3/3], Step [129300/138038], Loss: 2.5236, Perplexity: 12.4738
Epoch [3/3], Step [129400/1380

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.